<a href="https://colab.research.google.com/github/pleunipennings/CSC508_ML_Biomedicine_Class/blob/main/Module5/Module_5a_Adni_BoostedTrees_PatData_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Gradient-Boosted Trees notebook! 
This notebook was created at San Francisco State University for the PINC and gSTAR programs by Dr Pleuni Pennings, Lucy Moctezuma Tan and Lorena Benitez Rivera. We acknowledge help from Dr Adegoke Ojewole and Dr Hector Corrada Bravo from Genentech.

# Opening the file location and loading libraries

In [ ]:
# Below we are importing necessary libraries
import pandas as pd
import numpy as np

# Importing packages for Creating ML model and Evaluating it
from sklearn.model_selection import train_test_split 
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import preprocessing
from sklearn import metrics 
from sklearn.metrics import accuracy_score

# Importing library for plots
from matplotlib import pyplot as plt
from xgboost import plot_tree
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

Read the dataset "PatData_cleaned.csv", this dataset is already cleaned, we have dropped all missing values and it should only contain the main diagnoses:

|Diagnosis|Meaning|
|---|---|
|NL|Cognitively normal|
|MCI|Mild Cognitive Impairment|
|Dementia|Person that has Alzheimer's Disease|

In [ ]:
# Reading cleaned Dataset from Github
url = "https://raw.githubusercontent.com/pleunipennings/CSC508Data/main/PatData_cleaned.csv"
data = pd.read_csv(url)
data.head()

In [ ]:
# checking counts for people with each type of diagnosis
data['DX'].value_counts()

##  Preparing Training Data and Creating Gradient Boosted Tree Model Object

Split the data in labels (the diagnosis) and features (the other columns).  
Every algorithm works a bit differently depending on how each package is written, which is why it is always important to be updated on changes of your more used packages. In this particular case we see that for the Gradient Boosted tree from sklearn package the labels need to be numeric. 

In [ ]:
# Separating labels from the general dataframe
labels = data["DX"]

# Creating a label encoder object
le = preprocessing.LabelEncoder()

# Fitting the label encoder into the labels columns
le.fit(data["DX"])

# Transforming the classes into numbers
labels_t = le.transform(data["DX"])

Below we can see our actual named Diagnosis, and then our transformed labels. As you can see now:
- Dementia = 0
- MCI = 1
- NL = 2 

In [ ]:
# Printing the classes we have
list(le.classes_)

In [ ]:
# Printing how labels got transformed
np.unique(labels_t)

Now for our features we will drop patient ID, because it does not help us make any predictions and we drop the diagnosis since that is our label. All other columns should be considered as predictor features.

In [ ]:
# Dropping unnecessary columns for our features
features = data.drop(columns=['PTID','DX'])

The next part we should be pretty familiar with at this point:
 - We will separate our training and testing datasets using the labels and features we have been stablished.
 - We create the ML model object in this case it is Our Gradient Boosted Trees

**NOTE:** Notice that I have set ahead of time a couple of hyperparameters for my Gradient Boosted Tree already, such as a **seed** (for reproducible results), **eval_metric** (metric used to measure error, in this case is: "merror"), **max_depth** (each tree created will have a max of 4 layers deep before getting to a leaf), **learning_rate** (How fast we want our model to learn), **n_estimators** (Number of trees created by our model per class)

In [ ]:
# As mentioned in the textbook, we use about 70-80% of our data as the training data and the rest as test data. In the code, 70% training and 30% test
features_train, features_test, labels_train, labels_test = train_test_split(features, labels_t, test_size=0.3, random_state=42) 

#Create a Gradient Boosted Tree
gbt = XGBClassifier(seed = 42, eval_metric="merror", max_depth=4, learning_rate=0.5, n_estimators=50)

## Training using validation Data Gradient Boosted Tree

In order to illustrate how each tree created in the Gradient Boosted Tree improves, I will use a small validation dataset from the training data everytime our model makes a new tree.

- **Validation Data:** Must come from training data because, our test set is reserved for our final evaluation of the model. We use it in our example below so that you can see behind the scenes how much each iteration of trees get better results. In our case we will use 20 percent of our training data for our validation test.

**NOTE:** We should never use the test data during training because that would bias our model. This would be like knowing ahead of time the exact questions for an exam and then scoring high!

In [ ]:
# Extracting validation data from training data
X_train, X_valid, Y_train, Y_valid = train_test_split(features_train, labels_train, test_size=0.2, random_state=42) 

# Validation set
validation_set = [(X_train,Y_train),(X_valid, Y_valid)]

# Train a gradient Boosted Tree with validation data
gbt.fit(features_train, labels_train , eval_set = validation_set)

The output above shows us 2 columns, the first shows us how our model is doing in 80% percent of our training data, whereas the the other column shows what our validation set (20% of training data) is doing at each iteration. Looking at both columns at the same time we can get an idea of whether we might be overfirring or not. Since we are using the **merror** (Multiclass classification Error Rate) as our metric, we want to see that both columns show a decrease in this metric. As you can see they both do.

**NOTE:** For Multiclass Classifications the Gradient Boosted Tree Model actually creates 50 trees for each class! Dementia, MCI and NL.

Below you can actually get a summary of all the trees that were created. There are a lot of trees so we wont print all of them considering there is (50 X 3 classes) 150 trees total!

In [ ]:
# creating a list of all the trees created
gbt_treelist = gbt.get_booster().get_dump()
# Getting total amount of trees from XGboost Classifier model
print(len(gbt_treelist))

Here we can for example look at the first 2 trees created. The results below shows you the index for each node it creates for both of the trees. For example the **root node [index = 0] for the first tree is [Entorhinal<3239.82129]**

In [ ]:
# Check the total amopunt of trees 
for tree in gbt_treelist[0:2]:
  print(tree)

## Visualizing one tree from our Gradient Boosted Trees Model

The text summary above looks so convoluted and ugly! Below we can visualize one of our trees from our Gradient Boosted Tree Model. Lets choose in this case to look at the very first tree. You can change the index of the tree by changing the argument **num_trees**. Notice that this tree is just laying on its side, the rootnode is on the left while the leaves are towards the right.

In [ ]:
# Making a graph for our very first tree in Gradient Boosted Tree model
fig, ax = plt.subplots(figsize=(20, 20))
xgb.plot_tree(gbt, num_trees=0, ax=ax, rankdir="LR")
plt.show()


### Task 1: looking at the trees. 

1. Try plotting 2 or 3 more Decisions trees created. Why do you think they have 4 or less layers? 

In [ ]:
# graph some trees here and answer your question

**Answer to task 1**

Recall however that one of the biggest difference between Random Forests and Gradient Boosted Trees is that not all trees have equal amount of say on the final decision. As each tree created in Gradient Boosted Tree model tries to take into account the errors from the previous one, the trees with the lowest errors should have more say than the ones with more errors.

## Evaluating our Gradient Boosted Tree

Below we will finally use our **TESTING DATA** to evaluate our model. Our Testing data has never been seen before by our model, so this evaluation would emulate how our model could perform once it is deployed. Below we will use our model to make predictions four out test data and the look at the first 10 predicted values

In [ ]:
#Predict the response for test dataset
labels_pred = gbt.predict(features_test)

# Look at the predicted values. 
print(labels_pred[:10])
#Compare with the real data from the test data set. 
print(labels_test[:10])

The way a Gradient Boosted tree predicts each of the classes is through calculating different probabilities for each class. In the output below you can see that each row constitutes one prediction. Within each prediction we see 3 numbers. If you sum all the three number you get 100%. The index with the highest number is the final label predicted. Below is an example for the first predicted label:

|DX:|Dementia|Mild Cognitive Impairment|Normal|
|---|---|---|---|
|Index|0|1|2|
|Probabilities|0.9004508|0.05835567|0.0411935|

In [ ]:
# getting the probabilities predicted for each class
preds_proba = gbt.predict_proba(features_test)
print(preds_proba)

### Task 2:
What are the 10 first diagnosis predicted by our Gradient Boosted Tree?

**Answer for Task 2**

Below we will be plotting a Confusion matrix to Check to Check how our Gradient Boosted Tree has performed. We will also be calculating it's accuracy.

In [ ]:
#Let's visualize how well the GBT does. 
print(metrics.confusion_matrix(labels_test, labels_pred))
plt2 = metrics.ConfusionMatrixDisplay.from_estimator(gbt, features_test, labels_test)
plt.grid(False)

In [ ]:
# We want to check the accuracy in predicting the test data to make sure the model is not overfitted to the training data
accuracy = accuracy_score(labels_test, labels_pred)
print("Accuracy: %.1f%%" % (accuracy * 100))

#Task 3: Boosted trees 

Now it's your turn to train a Gradient-boosted tree model and a Random Forest model, and see which one does better in terms of overall accuracy. 

To make sure your model is a bit different from what we did previously in this notebook, I want you to choose just two of the diagnosis categories (NL, MCI, Dementia). With just two categories, the accuracy may become better than what we had before.  

1. Create your smaller dataset with just two diagnosis categories.
2. Split label and features, split training and test. 
3. Fit your models (gradient boosted trees and random forest). 
4. Predict for your test data and calculate accuracies. 
5. Plot your results in a confusion matrix. 
6. Which of the model does better? Is it a big difference? Do you think that any hyperparameter tuning could improve them? 



In [ ]:
# Create a Random Forest Model, train and evaluate it

In [ ]:
# Create a Gradient Boosted Tree Model, train and evaluate it

**Answers for task 3**